In [3]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../code")
import algorithm_for_game
import read_data
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# for sioux falls, read all the files
net_file_name = '../TransportationNetworks/SiouxFalls/SiouxFalls_net.tntp'
num_nodes, num_arcs, arc_set, free_flow_time, capacity = read_data.report_net_data_tntp(net_file_name)
origin_node = [6 - 1, 10 - 1, 23 - 1]
dest_node = [1 - 1, 13 - 1, 18 - 1, 20 - 1]
trip_file_name = '../TransportationNetworks/SiouxFalls/SiouxFalls_trips.tntp'
demand = read_data.read_flow_data_trip_tntp(trip_file_name, origin_node)
bpr_params = [0.15, 4]
# solve the static model
shortest_path_per_node, _ = read_data.find_shortest_path(free_flow_time, origin_node, dest_node)
# convert shortest path to arc set
shortest_path = {}
for i in origin_node:
    path = shortest_path_per_node[i]
    for j in range(len(path) - 1):
        cur_link = (path[j], path[j + 1])
        if cur_link not in shortest_path.keys():
            shortest_path[cur_link] = [i]
        else:
            shortest_path[cur_link].append(i)

In [24]:
trust = {}
for i in origin_node:
    trust[i] = 1
report_threshold = 3
num_dangerous_reports = {}
for i, j in arc_set:
    # generate a random value
    rand_val = np.random.rand()
    # set num_dangerous_reports
    if rand_val < 0.5:
        num_dangerous_reports[i, j] = 0
    elif rand_val < 0.7:
        num_dangerous_reports[i, j] = 1
    elif rand_val < 0.8:
        num_dangerous_reports[i, j] = 2
    else:
        num_dangerous_reports[i, j] = 3
# assume no roads have been explored
explore = {}
for i, j in arc_set:
    explore[(i, j)] = 0
# set weight to encourage exploration
eta = 0.01

In [37]:
result = algorithm_for_game.get_recommended_route(arc_set, origin_node, dest_node, num_nodes, bpr_params, trust, demand, capacity,
                          free_flow_time, shortest_path, explore=explore, eta=eta, report_threshold=report_threshold,
                          num_dangerous_reports=num_dangerous_reports)

Solve the optimization model to get optimal recommendation with weight on exploration.
Add flow variance objective
{(3, 4): 4.714050418094993e-08, (4, 3): 2.584591624654994, (21, 19): 1.6702097277754593, (5, 4): 4.369660099133115e-08, (20, 23): 2.7170989374842846e-08, (21, 22): 9.491539036058896e-09, (3, 10): 1.1544901158482887e-08, (5, 1): 0.7459743624412256, (5, 7): 1.365136790649453, (22, 23): 2.227414539591146, (14, 13): 1.5549537944580803e-08, (0, 2): 1.929921815939303e-08, (8, 9): 2.0992355111689302e-08, (11, 2): 3.146912713188596e-08, (9, 8): 3.496704548475407, (9, 14): 3.179489697511674, (10, 3): 0.9420306418476373, (1, 0): 0.7459643599757921, (10, 9): 1.2017159956972656e-08, (17, 6): 4.25816761641902e-08, (17, 15): 2.4442341201049617e-08, (2, 11): 6.537397537403412e-08, (19, 18): 9.044767433437289e-09, (19, 21): 8.372683299820356e-09, (13, 14): 0.011999026495638308, (15, 17): 3.9413971271984907, (18, 16): 0.5265721933993285, (18, 19): 1.7776798160816618, (6, 17): 2.27720957557

In [26]:
result['planned_total_time']

267.87188860896913

In [27]:
result['planned_exploration_term']


5.65418437760738

In [38]:
result['recommended_route']


{5: {(5, 1, 0): 0.7460643599757921, (5, 7, 6, 17): 1.3647464940111371},
 9: {(9, 8, 4, 3, 2, 0): 2.5840115014747416,
  (9, 10, 3, 2, 0): 0.9272790717938357,
  (9, 10, 11, 12): 1.5373375116234296,
  (9, 8, 7, 6, 17): 0.9120929211132963,
  (9, 14, 18, 16, 15, 17): 0.5264721933993285,
  (9, 16, 15, 17): 1.3660604897872952,
  (9, 15, 17): 2.0489042089745664,
  (9, 14, 18, 19): 1.7776997452318515,
  (9, 14, 21, 19): 0.8752877108424141},
 22: {(22, 13, 10, 3, 2, 0): 0.014271508849575091,
  (22, 23, 12): 1.8585134225477873,
  (22, 23, 20, 19): 0.3690011170433589,
  (22, 13, 14, 21, 19): 0.011999016250315075,
  (22, 21, 19): 0.7830230006827301,
  (22, 21, 20, 19): 0.9906597396977175}}

In [40]:
# test current case in the game: all demand is 1, bpr parameters are 0, 1
bpr_params = [0, 1]
demand = {}
for i in origin_node:
    demand[i] = 1
result = algorithm_for_game.get_recommended_route(arc_set, origin_node, dest_node, num_nodes, bpr_params, trust, demand, capacity,
                          free_flow_time, shortest_path, explore=explore, eta=eta, report_threshold=report_threshold,
                          num_dangerous_reports=num_dangerous_reports)

Solve the optimization model to get optimal recommendation with weight on exploration.
Add flow variance objective
{(3, 4): 9.710413281203141e-07, (4, 3): 1.1876764496989571e-05, (21, 19): 1.6747595718274057e-06, (5, 4): 2.2167230756537962e-05, (20, 23): 5.993227814330694e-07, (21, 22): 3.8057320345975847e-07, (3, 10): 5.051900723336859e-07, (5, 1): 1.2150460888785504e-05, (5, 7): 0.9999672272697319, (22, 23): 0.9999752711702627, (14, 13): 5.753381808812075e-07, (0, 2): 6.972930213166995e-07, (8, 9): 6.059998808042204e-07, (11, 2): 7.66074056608271e-07, (9, 8): 1.1800045245643434e-05, (9, 14): 1.2714710281951464e-05, (10, 3): 6.705261522792402e-07, (1, 0): 1.2218772032732817e-06, (10, 9): 3.4861685213548294e-07, (17, 6): 7.613981110907064e-07, (17, 15): 5.297285722772101e-07, (2, 11): 1.0327149417324084e-06, (19, 18): 1.0438397724022963e-06, (19, 21): 4.609511126581964e-07, (13, 14): 8.849072430257006e-07, (15, 17): 0.9999368278717049, (18, 16): 9.407782739295373e-07, (18, 19): 1.03600

In [41]:
result['planned_total_time']


19.999851153449065

In [42]:
result['planned_exploration_term']


0.1958968199940397

In [46]:
result['recommended_route']
# this matches the shortest path

{5: {(5, 7, 6, 17): 0.9998275037095343},
 9: {(9, 15, 17): 0.9995963407785751},
 22: {(22, 23, 12): 1.0}}